In [80]:
import pandas as pd

In [81]:
test_results_file = "../experiments/task_detect_xss_simple_prompt/experiments_summary_test.csv"
test_synth_results_file = "../experiments/task_detect_xss_simple_prompt/test_results_synth.csv"

RQ1

In [82]:
df = pd.read_csv(test_results_file)
df.head()

,Unnamed: 0,successes,failures,total,accuracy,accuracy_std,accuracy_var,precision,precision_std,precision_var,...,top_15_accuracy_var,top_15_precision,top_15_precision_std,top_15_precision_var,top_15_recall,top_15_recall_std,top_15_recall_var,top_15_f1,top_15_f1_std,top_15_f1_var
0,0,39,1,40,0.608720,0.154749,2.394734e-02,0.999486,0.003170,0.000010,...,2.379489e-02,0.998663,0.005002,0.000025,0.519734,3.147747e-01,9.908310e-02,0.616461,3.260624e-01,1.063167e-01
1,1,40,0,40,0.513053,0.000041,1.697392e-09,1.000000,0.000000,0.000000,...,4.129329e-09,1.000000,0.000000,0.000000,0.012321,1.303430e-04,1.698931e-08,0.024342,2.543084e-04,6.467275e-08
2,2,40,0,40,0.567779,0.116080,1.347448e-02,0.962351,0.173155,0.029983,...,2.207001e-02,1.000000,0.000000,0.000000,0.311912,3.013349e-01,9.080270e-02,0.403234,3.156556e-01,9.963847e-02
3,3,40,0,40,0.548601,0.094178,8.869554e-03,0.989746,0.044743,0.002002,...,1.803282e-02,0.972655,0.069793,0.004871,0.236145,3.331036e-01,1.109580e-01,0.273386,3.286533e-01,1.080130e-01
4,4,40,0,40,0.513043,0.000000,0.000000e+00,1.000000,0.000000,0.000000,...,0.000000e+00,1.000000,0.000000,0.000000,0.012270,3.469447e-18,1.203706e-35,0.024242,6.938894e-18,4.814825e-35


In [83]:
list(df.columns)

['Unnamed: 0',
 'successes',
 'failures',
 'total',
 'accuracy',
 'accuracy_std',
 'accuracy_var',
 'precision',
 'precision_std',
 'precision_var',
 'recall',
 'recall_std',
 'recall_var',
 'f1',
 'f1_std',
 'f1_var',
 'run',
 'data',
 'function_name',
 'isolated_execution',
 'summarize_results',
 'result_file_name',
 'create_confusion_matrix',
 'top_k_metric',
 'top_k',
 'model_name',
 'temperature',
 'task',
 'template',
 'prompt_parameters',
 'generation_mode',
 'experiments_folder',
 'experiments',
 'parameters_file_name',
 'input_prompt_file_name',
 'hf_max_new_tokens',
 'hf_load_in_4bit',
 'seed',
 'example_template',
 'examples_per_class',
 'examples_file',
 'examples_payload_column',
 'examples_label_column',
 'example_positive_label',
 'example_negative_label',
 'rag_template_file',
 'rag_source',
 'db_persist_path',
 'chunk_size',
 'chunk_overlap',
 'folder',
 'top_1_experiments',
 'top_1_accuracy',
 'top_1_accuracy_std',
 'top_1_accuracy_var',
 'top_1_precision',
 'top_1_pr

In [84]:
examples_values = set(df["examples_per_class"].values.tolist())
examples_values

{0, 1, 3, 5}

In [85]:
new_columns = ["model_temperature"]
new_columns.extend(list(map(lambda x:f"no_rag_{x}",examples_values)))
new_columns.extend(list(map(lambda x:f"rag_{x}",examples_values)))
new_columns

['model_temperature',
 'no_rag_0',
 'no_rag_1',
 'no_rag_3',
 'no_rag_5',
 'rag_0',
 'rag_1',
 'rag_3',
 'rag_5']

In [86]:
new_df = pd.DataFrame(columns=new_columns)

In [87]:
df["model_temperature"] = df["model_name"]+"_"+df["temperature"].astype(str)

In [88]:
#order df by model_temperature
df = df.sort_values(by=["model_temperature"])

In [89]:
for model_temperature in df["model_temperature"].unique():
    print(model_temperature)
    df_model_temperature = df[df["model_temperature"]==model_temperature]
    print(len(df_model_temperature))
    new_row = {"model_temperature":model_temperature}
    for examples_per_class in examples_values:
        if int(examples_per_class) == 0:
            # generation mode zero shot and examples per class = 0
            new_row[f"no_rag_{examples_per_class}"] = df_model_temperature[(df_model_temperature["generation_mode"]=="zero_shot") & (df_model_temperature["examples_per_class"]==0)]["accuracy"].values[0]
            # generation mode rag and examples per class = 0
            new_row[f"rag_{examples_per_class}"] = df_model_temperature[(df_model_temperature["generation_mode"]=="rag") & (df_model_temperature["examples_per_class"]==0)]["accuracy"].values[0]
        else:
            # generation mode few_shot and examples per class = examples per class
            new_row[f"no_rag_{examples_per_class}"] = df_model_temperature[(df_model_temperature["generation_mode"]=="few_shot") & (df_model_temperature["examples_per_class"]==examples_per_class)]["accuracy"].values[0]
            # generation mode rag_few_shot and examples per class = examples per class
            new_row[f"rag_{examples_per_class}"] = df_model_temperature[(df_model_temperature["generation_mode"]=="rag_few_shot") & (df_model_temperature["examples_per_class"]==examples_per_class)]["accuracy"].values[0]
    new_df = pd.concat([new_df,pd.DataFrame(new_row,index=[0])])

gpt-3.5-turbo-0125_0.0
8
gpt-3.5-turbo-0125_0.5
8
gpt-3.5-turbo-0125_1.0
8
gpt-3.5-turbo-0613_0.0
8
gpt-3.5-turbo-0613_0.5
8
gpt-3.5-turbo-0613_1.0
8
gpt-4-0125-preview_0.0
8
gpt-4-0125-preview_0.5
8
gpt-4-0125-preview_1.0
8
gpt-4-1106-preview_0.0
8
gpt-4-1106-preview_0.5
8
gpt-4-1106-preview_1.0
8


/tmp/ipykernel_3142293/2230646587.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df = pd.concat([new_df,pd.DataFrame(new_row,index=[0])])


In [90]:
new_df.head(12)

,model_temperature,no_rag_0,no_rag_1,no_rag_3,no_rag_5,rag_0,rag_1,rag_3,rag_5
0,gpt-3.5-turbo-0125_0.0,0.506994,0.544594,0.617817,0.693261,0.509485,0.513043,0.513053,0.521564
0,gpt-3.5-turbo-0125_0.5,0.505440,0.588013,0.656266,0.651586,0.568305,0.548601,0.608720,0.525326
0,gpt-3.5-turbo-0125_1.0,0.505349,0.666718,0.668919,0.707629,0.610274,0.623215,0.567779,0.599898
0,gpt-3.5-turbo-0613_0.0,0.513043,0.528034,0.696408,0.554631,0.513043,0.513043,0.513043,0.521564
0,gpt-3.5-turbo-0613_0.5,0.621092,0.668732,0.608306,0.719849,0.567466,0.548601,0.565600,0.529494
0,gpt-3.5-turbo-0613_1.0,0.654555,0.718763,0.663716,0.689454,0.632770,0.629912,0.616435,0.545307
0,gpt-4-0125-preview_0.0,0.603648,0.575983,0.748086,0.782555,0.813426,0.817357,0.813951,0.919749
0,gpt-4-0125-preview_0.5,0.648204,0.590992,0.700213,0.742911,0.866938,0.882963,0.764546,0.874097
0,gpt-4-0125-preview_1.0,0.640381,0.737270,0.751505,0.763710,0.770027,0.760521,0.813994,0.887216
0,gpt-4-1106-preview_0.0,0.662977,0.754471,0.732277,0.801418,0.794469,0.826077,0.838464,0.867506


RQ2

In [91]:
df_synth = pd.read_csv(test_synth_results_file)
df_synth.head()

,Unnamed: 0,top_k,dataset,accuracy,accuracy_diff,dataset_index,task,template,prompt_parameters,model_name,generation_mode,examples_per_class,temperature,seed
0,0,1,gpt-3.5-turbo-0125_1.0_rag_few_shot_3,0.977505,0.002079,0,detect_xss_simple_prompt,create_function_readable,empty,gpt-3.5-turbo-0125,rag_few_shot,3,0.5,156
1,1,1,gpt-3.5-turbo-0125_1.0_rag_few_shot_3,0.834594,0.144991,1,detect_xss_simple_prompt,create_function_readable,empty,gpt-3.5-turbo-0125,rag_few_shot,3,0.5,156
2,2,1,gpt-3.5-turbo-0125_1.0_rag_few_shot_3,0.977505,0.002079,2,detect_xss_simple_prompt,create_function_readable,empty,gpt-3.5-turbo-0125,rag_few_shot,3,0.5,156
3,3,1,gpt-3.5-turbo-0125_1.0_rag_few_shot_3,0.513043,0.466541,3,detect_xss_simple_prompt,create_function_readable,empty,gpt-3.5-turbo-0125,rag_few_shot,3,0.5,156
4,4,1,gpt-3.5-turbo-0125_1.0_rag_few_shot_3,0.836484,0.143100,4,detect_xss_simple_prompt,create_function_readable,empty,gpt-3.5-turbo-0125,rag_few_shot,3,0.5,156


In [92]:
list(df_synth.columns)

['Unnamed: 0',
 'top_k',
 'dataset',
 'accuracy',
 'accuracy_diff',
 'dataset_index',
 'task',
 'template',
 'prompt_parameters',
 'model_name',
 'generation_mode',
 'examples_per_class',
 'temperature',
 'seed']

In [93]:
df_synth["experiment"] = df_synth["model_name"]+"_"+df_synth["temperature"].astype(str) + "_"+df_synth["generation_mode"]+"_"+df_synth["examples_per_class"].astype(str)

In [94]:
best_exp = "gpt-4-0125-preview_0.0_rag_few_shot_5"
worst_case = "gpt-4-0125-preview_1.0_zero_shot_0"
middle_case = "gpt-4-0125-preview_0.5_few_shot_3"


In [95]:
top_ks = set(df_synth["top_k"].values.tolist())
top_ks

{1, 3, 5, 10, 15}

In [96]:
new_columns = ["experiment", "dataset"]
new_columns.extend(list(map(lambda x:f"top_{x}_acc_diff",top_ks)))
new_columns.extend(list(map(lambda x:f"top_{x}_acc",top_ks)))
new_columns

['experiment',
 'dataset',
 'top_1_acc_diff',
 'top_3_acc_diff',
 'top_5_acc_diff',
 'top_10_acc_diff',
 'top_15_acc_diff',
 'top_1_acc',
 'top_3_acc',
 'top_5_acc',
 'top_10_acc',
 'top_15_acc']

In [97]:
experiments_to_keep = [best_exp,worst_case,middle_case]
new_df_synth = pd.DataFrame(columns=new_columns)


In [98]:
df_synth.dataset.unique()

array(['gpt-3.5-turbo-0125_1.0_rag_few_shot_3',
       'gpt-3.5-turbo-0125_0.5_zero_shot',
       'gpt-3.5-turbo-0125_1.0_zero_shot',
       'gpt-3.5-turbo-0125_1.0_few_shot_3',
       'gpt-3.5-turbo-0125_0.5_few_shot_1',
       'gpt-3.5-turbo-0125_1.0_few_shot_1',
       'gpt-3.5-turbo-0125_1.0_few_shot_5',
       'gpt-4-0125-preview_0.5_rag_few_shot_3',
       'gpt-4-0125-preview_1.0_rag_few_shot_3',
       'gpt-4-0125-preview_0.5_rag_few_shot_1',
       'gpt-4-0125-preview_0.0_rag_few_shot_1',
       'gpt-4-0125-preview_1.0_rag_few_shot_1',
       'gpt-4-0125-preview_0.5_rag_few_shot_5',
       'gpt-4-0125-preview_1.0_rag_few_shot_5',
       'gpt-4-0125-preview_0.5_zero_shot',
       'gpt-4-0125-preview_1.0_zero_shot',
       'gpt-4-0125-preview_0.5_few_shot_3',
       'gpt-4-0125-preview_1.0_few_shot_3',
       'gpt-4-0125-preview_0.5_few_shot_1',
       'gpt-4-0125-preview_1.0_few_shot_1',
       'gpt-4-0125-preview_0.5_few_shot_5',
       'gpt-4-0125-preview_0.0_few_shot_5',
    

In [99]:
def from_dataset_to_splits(row):
   dataset = row["dataset"]
   #check in dataset ends with zero_shot or rag
   if dataset.endswith("zero_shot") or dataset.endswith("rag"):
      dataset = dataset +"_0"
   #print(dataset)
   parts = dataset.split("_")
   generation = "_".join(parts[2:-1])
   row["dataset_model"] = parts[0]
   row["dataset_temperature"] = parts[1]
   row["dataset_generation_mode"] = 0 if generation == "zero_shot" or generation == "few_shot" else 1
   row["dataset_examples_per_class"] = parts[3]
   return row


In [100]:
for experiment in experiments_to_keep:
    df_keep = df_synth[df_synth["experiment"]==experiment]
    df_keep = df_keep.apply(from_dataset_to_splits,axis=1)

    df_keep = df_keep.sort_values(by=["dataset_model", "dataset_temperature", "dataset_generation_mode", "dataset_examples_per_class"])
    for dataset in df_keep["dataset"].unique():
        df_dataset = df_keep[df_keep["dataset"]==dataset]
        new_row = {"experiment":experiment,"dataset":dataset}
        #group by top_k and avg the accuracy and acc_diff
        for top_k in top_ks:
            df_top = df_dataset[df_dataset["top_k"]==top_k]
            new_row[f"top_{top_k}_acc_diff"] = df_top["accuracy_diff"].mean()
            new_row[f"top_{top_k}_acc"] = df_top["accuracy"].mean()
        new_df_synth = pd.concat([new_df_synth,pd.DataFrame(new_row,index=[0])])

 


/tmp/ipykernel_3142293/219162767.py:14: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  new_df_synth = pd.concat([new_df_synth,pd.DataFrame(new_row,index=[0])])


In [101]:
new_df_synth.head()

,experiment,dataset,top_1_acc_diff,top_3_acc_diff,top_5_acc_diff,top_10_acc_diff,top_15_acc_diff,top_1_acc,top_3_acc,top_5_acc,top_10_acc,top_15_acc
0,gpt-4-0125-preview_0.0_rag_few_shot_5,gpt-3.5-turbo-0125_0.5_zero_shot,0.020038,0.031456,0.031520,0.053940,0.042897,0.958790,0.947372,0.947308,0.924756,0.926176
0,gpt-4-0125-preview_0.0_rag_few_shot_5,gpt-3.5-turbo-0125_0.5_few_shot_1,0.012325,0.038815,0.043319,0.082849,0.068218,0.966503,0.940013,0.935509,0.895847,0.900856
0,gpt-4-0125-preview_0.0_rag_few_shot_5,gpt-3.5-turbo-0125_1.0_zero_shot,0.041701,0.040340,0.036503,0.053151,0.040571,0.937127,0.938488,0.942325,0.925544,0.928503
0,gpt-4-0125-preview_0.0_rag_few_shot_5,gpt-3.5-turbo-0125_1.0_few_shot_3,0.016011,0.026043,0.038367,0.054149,0.049690,0.962817,0.952785,0.940461,0.924546,0.919384
0,gpt-4-0125-preview_0.0_rag_few_shot_5,gpt-3.5-turbo-0125_1.0_few_shot_1,0.036522,0.082867,0.070809,0.073524,0.059070,0.942306,0.895961,0.908019,0.905172,0.910004


In [103]:
len(new_df_synth.dataset.unique())

26